In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re
import string
import nltk 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.pipeline import Pipeline

%matplotlib inline 

In [2]:
df = pd.read_csv("olid-training-v1.0.csv",delimiter="\t")#导入数据集
df

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [3]:
#去除评论中的表情包
def demoji(text):
        emoji_pattern = re.compile("["
                                  u"\U0001F600-\U0001F64F"
                                  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                  u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                  u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                  u"\U00002702-\U000027B0"
                                  u"\U000024C2-\U0001F251"
                                  u"\U00010000-\U0010ffff"
                                   "]+", flags=re.UNICODE)
        return(emoji_pattern.sub(r'', text))
                                  
     
                                  
   
 

df[u'tweet'] = df[u'tweet'].astype(str)
df[u'tweet'] = df[u'tweet'].apply(lambda x:demoji(x))

In [4]:
training = df.tweet

In [5]:
training

0        @USER She should ask a few native Americans wh...
1        @USER @USER Go home you’re drunk!!! @USER #MAG...
2        Amazon is investigating Chinese employees who ...
3        @USER Someone should'veTaken" this piece of sh...
4        @USER @USER Obama wanted liberals &amp; illega...
                               ...                        
13235    @USER Sometimes I get strong vibes from people...
13236    Benidorm   Creamfields   Maga    Not too shabb...
13237    @USER And why report this garbage.  We don't g...
13238                                          @USER Pussy
13239    #Spanishrevenge vs. #justice #HumanRights and ...
Name: tweet, Length: 13240, dtype: object

In [6]:
for i in range(len(training)):## delete all @USER
    training[i] = training[i].replace("@USER","")

D:\Application\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
training

0         She should ask a few native Americans what th...
1           Go home you’re drunk!!!  #MAGA #Trump2020  URL
2        Amazon is investigating Chinese employees who ...
3         Someone should'veTaken" this piece of shit to...
4          Obama wanted liberals &amp; illegals to move...
                               ...                        
13235     Sometimes I get strong vibes from people and ...
13236    Benidorm   Creamfields   Maga    Not too shabb...
13237     And why report this garbage.  We don't give a...
13238                                                Pussy
13239    #Spanishrevenge vs. #justice #HumanRights and ...
Name: tweet, Length: 13240, dtype: object

In [8]:
for i in range(len(training)):## lowercase
    training[i] = training[i].lower()

D:\Application\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
training

0         she should ask a few native americans what th...
1           go home you’re drunk!!!  #maga #trump2020  url
2        amazon is investigating chinese employees who ...
3         someone should'vetaken" this piece of shit to...
4          obama wanted liberals &amp; illegals to move...
                               ...                        
13235     sometimes i get strong vibes from people and ...
13236    benidorm   creamfields   maga    not too shabb...
13237     and why report this garbage.  we don't give a...
13238                                                pussy
13239    #spanishrevenge vs. #justice #humanrights and ...
Name: tweet, Length: 13240, dtype: object

In [10]:
#去除文本当中的标点符号
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
training = training.apply(lambda x: remove_punct(x))

In [11]:
## 把文本中的大写字母改为小写
for i in range(len(training)):
    training[i] = training[i].replace("url","")

In [12]:
#分割所有的单词成为数组，单词作为元素
def tokenization(text):
    text = re.split('\W+', text)
    return text

training_token = training.apply(lambda x: tokenization(x.lower()))


In [13]:
training_token

0        [, she, should, ask, a, few, native, americans...
1              [, go, home, you, re, drunk, maga, trump, ]
2        [amazon, is, investigating, chinese, employees...
3        [, someone, shouldvetaken, this, piece, of, sh...
4        [, obama, wanted, liberals, amp, illegals, to,...
                               ...                        
13235    [, sometimes, i, get, strong, vibes, from, peo...
13236    [benidorm, creamfields, maga, not, too, shabby...
13237    [, and, why, report, this, garbage, we, dont, ...
13238                                            [, pussy]
13239    [spanishrevenge, vs, justice, humanrights, and...
Name: tweet, Length: 13240, dtype: object

In [14]:
#下载stopwords，无意义的she ,he , is, the 可以去掉
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LeoFishLiao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
stopword = set(stopwords.words('english'))

In [16]:
stopword

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [17]:
#去除无意义的单词
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
training_stop = training.apply(lambda x: remove_stopwords(x))


In [18]:
#把前面的所有数据清理放进一个函数当中
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

In [19]:
#数据的向量化，然后对其进行归一化
countVectorizer = CountVectorizer(analyzer=clean_text)
countVector = countVectorizer.fit_transform(training)

In [20]:
countVector

<13240x19925 sparse matrix of type '<class 'numpy.int64'>'
	with 143475 stored elements in Compressed Sparse Row format>

In [21]:
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

13240 Number of tweets has 19925 words


In [22]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
# count_vect_df.to_csv('vector.csv')

In [23]:
count_vect_df

,,aa,aaa,aaaaaaaaa,aaaah,aaaahh,aalayahi,aand,aaron,aarp,...,zuckerberg,³,älskar,òne,ˢᵗᵉᵖ,ᴳᵉᵗᵗᶦⁿᵍ,ᴵⁿᵈᵉˣ,ᵗʰᵉ,ᶠᶦʳˢᵗ,ᶦˢ
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13235,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13237,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13238,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
training

0         she should ask a few native americans what th...
1                       go home you’re drunk  maga trump  
2        amazon is investigating chinese employees who ...
3         someone shouldvetaken this piece of shit to a...
4          obama wanted liberals amp illegals to move i...
                               ...                        
13235     sometimes i get strong vibes from people and ...
13236    benidorm   creamfields   maga    not too shabb...
13237     and why report this garbage  we dont give a crap
13238                                                pussy
13239    spanishrevenge vs justice humanrights and free...
Name: tweet, Length: 13240, dtype: object

In [25]:
labels = df.subtask_a	

In [26]:
labels

0        OFF
1        OFF
2        NOT
3        OFF
4        NOT
        ... 
13235    OFF
13236    NOT
13237    OFF
13238    OFF
13239    NOT
Name: subtask_a, Length: 13240, dtype: object

In [27]:
#使用多分类朴素贝叶斯

In [28]:
#设计一个pipeline包含向量化和分类器
text_clf = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(1,2))),
        ('clf', MultinomialNB()),])


In [29]:
text_clf.fit(training,labels)#拟合数据

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x0000024D448240D8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [30]:
from sklearn.model_selection import cross_val_score
# clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(text_clf, training, labels, cv=5)

In [31]:
scores

array([0.74811178, 0.74962236, 0.75490937, 0.73904834, 0.76435045])

In [32]:
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [33]:
test = pd.read_csv('testset-levela.csv',delimiter = "\t").tweet

In [34]:
predicted = text_clf.predict(test)

In [35]:
test_label = pd.read_csv('labels-levela.csv')

In [36]:
test_label = test_label.Label

In [37]:
np.mean(predicted == test_label)

0.791860465116279

In [38]:
#使用其他分类器-------支持向量机

In [39]:
from sklearn.svm import LinearSVC

In [40]:
text_clf2 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,4))),
        ('tfidf', TfidfTransformer()),#generalization
        ('clf', LinearSVC())])

In [41]:
text_clf2.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x0000024D448240D8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept

In [42]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.74811178, 0.74962236, 0.75490937, 0.73904834, 0.76435045])

In [43]:
predicted = text_clf2.predict(test)

In [44]:
np.mean(predicted == test_label)

0.7802325581395348

In [45]:
##使用随机森林

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
text_clf3 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,2))),
        ('clf',RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0))])

In [48]:
text_clf3.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x0000024D448240D8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                ('clf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=3,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
         

In [49]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.74811178, 0.74962236, 0.75490937, 0.73904834, 0.76435045])

In [50]:
predicted = text_clf3.predict(test)

In [51]:
np.mean(predicted == test_label)

0.7209302325581395

In [52]:
#使用Xgboost

In [53]:
!pip install xgboost

In [54]:
import xgboost as xgb

In [55]:
text_clf4 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text,ngram_range=(2,4))),
        ('clf',xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1))])

In [56]:
text_clf4.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x0000024D448240D8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                               interaction_constraints=None, learning_rate=0.1,
                               max_delta_step=0, max_depth=7,
                               min_child_weight=1, missing=nan,
                               monotone_constraints=None, n_estimators=200,
                               n_jobs=10, nthread=10, num_parallel_tree=1,
           

In [57]:
scores = cross_val_score(text_clf, training, labels, cv=5)
scores

array([0.74811178, 0.74962236, 0.75490937, 0.73904834, 0.76435045])

In [58]:
predicted = text_clf4.predict(test)

In [59]:
np.mean(predicted == test_label)

0.8058139534883721

In [60]:
pre = pd.Series(predicted,name="predicted")
lab = pd.Series(test_label)
pd.DataFrame([test,lab,pre]).T.loc[200:210]

,tweet,Label,predicted
200,#Antifa demo in memory of #KillahP. #Greece URL,NOT,NOT
201,@USER These are all coranated attacks from id...,OFF,OFF
202,@USER @USER Awwww she is so stinking cute! How...,OFF,NOT
203,#Benghazi #Haiti #UraniumOne #SethRich What...,OFF,NOT
204,"#Black_Eyed_Peas Tackle Gun Control, Social Ju...",NOT,NOT
205,#Censorship Awake yet? Ask yourself why thes...,NOT,NOT
206,You didnt have to beat him up like that... all...,NOT,NOT
207,#JoinTheFight Join the fight against Americ...,OFF,OFF
208,#Art #ArtistsOnTwitter #Soul #GoodBusiness #En...,NOT,NOT
209,#ICYMI: Welcome Anna Grey Barbour to our Women...,NOT,NOT


In [61]:
##使用SDG

In [62]:
text_clf5 = Pipeline([
        ('conuntVectorizer', CountVectorizer(analyzer=clean_text)),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None))])

In [63]:
text_clf5.fit(training,labels)

Pipeline(memory=None,
         steps=[('conuntVectorizer',
                 CountVectorizer(analyzer=<function clean_text at 0x0000024D448240D8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=5, n_iter_no_c

In [64]:
predicted = text_clf5.predict(test)

In [65]:
np.mean(predicted == test_label)

0.7534883720930232

In [66]:
pre = pd.Series(predicted,name="predicted")
lab = pd.Series(test_label)
pd.DataFrame([test,lab,pre]).T.loc[200:210]

,tweet,Label,predicted
200,#Antifa demo in memory of #KillahP. #Greece URL,NOT,NOT
201,@USER These are all coranated attacks from id...,OFF,NOT
202,@USER @USER Awwww she is so stinking cute! How...,OFF,NOT
203,#Benghazi #Haiti #UraniumOne #SethRich What...,OFF,NOT
204,"#Black_Eyed_Peas Tackle Gun Control, Social Ju...",NOT,NOT
205,#Censorship Awake yet? Ask yourself why thes...,NOT,NOT
206,You didnt have to beat him up like that... all...,NOT,NOT
207,#JoinTheFight Join the fight against Americ...,OFF,NOT
208,#Art #ArtistsOnTwitter #Soul #GoodBusiness #En...,NOT,NOT
209,#ICYMI: Welcome Anna Grey Barbour to our Women...,NOT,NOT
